In [46]:
import pickle
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [47]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
df = pd.read_csv('modified_data.csv', encoding='ISO-8859-1')

In [50]:
df.head()

,Unnamed: 0,target,id,date,flag,user,text,stemmed_content
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [51]:
twitter_data = pd.read_excel('tweets1.xlsx', names=['tweets'])
twitter_data.head()

,tweets
0,Ready to embrace the slimy adventure? Share yo...
1,This is why we #bitcoin
2,Motokos are coming to Bitcoin Ordinals! All as...
3,A family member called and said: you should co...
4,"JUST IN: Fidelity, the world's third-largest a..."


In [52]:
#stemming
port_stem = PorterStemmer()

def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [53]:
twitter_data['stemmed_tweets'] = twitter_data['tweets'].apply(stemming)

In [54]:
twitter_data.head()

,tweets,stemmed_tweets
0,Ready to embrace the slimy adventure? Share yo...,readi embrac slimi adventur share excit chanc ...
1,This is why we #bitcoin,bitcoin
2,Motokos are coming to Bitcoin Ordinals! All as...,motoko come bitcoin ordin asset fulli inscrib ...
3,A family member called and said: you should co...,famili member call said consid sell bitcoin pr...
4,"JUST IN: Fidelity, the world's third-largest a...",fidel world third largest asset manag file spo...


In [67]:
#model will train with the stemmed_content and sentiment data

stemmed_content = df['stemmed_content'].values
sentiment = df['target'].values
stemmed_tweets = twitter_data['stemmed_tweets'].values

In [68]:
#converting to numerical data
vectorizer = TfidfVectorizer()

stemmed_content = vectorizer.fit_transform(stemmed_content.astype('U'))
stemmed_tweets = vectorizer.transform(stemmed_tweets.astype('U'))

In [71]:
print(stemmed_tweets)

  (0, 511609)	0.2014493271562646
  (0, 511215)	0.14806239262396897
  (0, 475185)	0.16349708336822796
  (0, 454488)	0.19997131820935704
  (0, 438342)	0.1986092812645946
  (0, 429484)	0.21408785268798614
  (0, 429427)	0.2822826635741988
  (0, 418400)	0.16530459240273213
  (0, 391354)	0.19399975285968496
  (0, 386406)	0.13365541338418171
  (0, 350435)	0.21102578125447402
  (0, 271004)	0.09273560472176315
  (0, 268654)	0.1270606185742737
  (0, 234804)	0.18386735930435708
  (0, 232030)	0.16338184258075175
  (0, 206988)	0.22192661873316336
  (0, 183190)	0.32292200160876233
  (0, 145027)	0.23123432161903967
  (0, 144958)	0.13638283388258424
  (0, 136598)	0.23966574184360323
  (0, 76922)	0.33656106741884567
  (0, 45700)	0.19595564356902753
  (0, 33317)	0.13652331882788704
  (0, 5636)	0.1996520575726155
  (2, 493081)	0.1276912454220447
  :	:
  (22, 196471)	0.2601948545970114
  (22, 116445)	0.34980491589000556
  (22, 52519)	0.3797940309167975
  (23, 462103)	0.12160571136189788
  (23, 446735)	0.1

In [72]:
#Logistic regression ML model

model = LogisticRegression(max_iter=1000)

In [73]:
model.fit(stemmed_content, sentiment)

LogisticRegression(max_iter=1000)

In [74]:
stemmed_tweets_prediction = model.predict(stemmed_tweets)
print(stemmed_tweets_prediction)

[1 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 1]


In [76]:
popular_crypto = ['bitcoin', 'ethereum', 'tether USDT', 'BNB', 'solana']
for i in range(5):
    print("%s : %i" % (popular_crypto[i], (stemmed_tweets_prediction[5*i] + stemmed_tweets_prediction[5*i + 1] + stemmed_tweets_prediction[5*i + 2] + stemmed_tweets_prediction[5*i + 3])))

bitcoin : 2
ethereum : 4
tether USDT : 1
BNB : 2
solana : 3


In [77]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))